# IMPORT MAIN PACKAGES

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import gc
# Read in shapefile and examine data
fp = "C:/Users/OLYMPIA/Europe/E1/CNTR_RG_01M_2016_4326.shp"
fk = "C:/Users/OLYMPIA/Europe/E2/CNTR_BN_01M_2016_4326.shp"
fi = "C:/Users/OLYMPIA/Europe/I1/it_100km.shp"
fe = "C:/Users/OLYMPIA/Europe/E3/Europe_coastline_poly.shp"
fa = "C:/Users/OLYMPIA/Europe/E3/Europe_coastline_poly.shp"
fz = "C:/Users/OLYMPIA/Europe/E4/NUTS_BN_01M_2016_4326.shp"


# Import Shapely files for geoplotting and dataset related to selection processes


In [2]:
euro = gpd.read_file(fp)

In [3]:
Super_euro = euro[euro['NAME_ENGL'].isin(['Italy', 'France', 'Spain', 'Austria', 'Switzerland', 'Germany', 'Denmark', 
                                           'Netherlands', 'Belgium', 'Luxembourg', 'Portugal', 'United Kingdom',
                                           'Ireland', 'Slovenia', 'Czechia', 'Hungary', 'Croatia', 'Slovakia',
                                            'Bulgaria','Bosnia and Herzegovina', 'Montenegro', 
                                           'Serbia', 'North Macedonia', 'Albania', 'Greece', 'Andorra', 'Liechtenstein'])]

#'Romania'
#'Poland'

In [4]:
job_sea_20 = pd.ExcelFile(r'C:\Users\OLYMPIA\Desktop\Personal Andrea\ASSEGNO LAVORO\Report Job Search 2019\Job_sea_2020_Geo.xlsx')
job_sea_20 = job_sea_20.parse('Research', encoding='latin_1')
df3 = job_sea_20.loc[:, ['Id', 'Date_first', 'State', 'City', 'Position', 'Job_Families', 'Latitude', 'Longitude',
                         'Source','Reason']]

# Enrich SUPER_EURO Dataset by adding the number of processes

In [5]:
def F(row,col_name): ### if State is Italy, Spain, Austria, Germany, Denmark, France                                                             
    if row[col_name] == 'Italy': 
        val = 39
    elif row[col_name] == 'Spain':
        val = 14
    elif row[col_name] == 'Austria':
        val = 9
    elif row[col_name] == 'Denmark':
        val = 3
    elif row[col_name] == 'Germany':
        val = 1
    elif row[col_name] == 'France':
        val = 1
    else:
        val = 0

    return val

In [6]:
df2 = Super_euro.loc[:, ['NAME_ENGL']]
df2['Nu_Proc'] = df2.apply(lambda row: F(row,'NAME_ENGL') , axis=1)
Super_euro = pd.concat([Super_euro, df2['Nu_Proc']], axis=1)
del df2
gc.collect()

543

In [7]:
df3['Nu_Proc_Loc'] = df3.apply(lambda x: 1, axis=1)
df_euro = df3.groupby('City').agg({ 'Nu_Proc_Loc': np.sum})
df_eu = df3.loc[:, ['City', 'Latitude', 'Longitude']]
df_eu = df_eu.drop_duplicates()
df_euro = df_euro.reset_index()
pro_sel = pd.merge(df_eu, df_euro, how='inner', left_on = 'City', right_on = 'City')

In [8]:
pro_sel['Size_Scale'] = (pro_sel['Nu_Proc_Loc'])*2

In [9]:
Super_euro.drop('CNTR_ID', axis=1, inplace=True)
Super_euro.drop('CNTR_NAME', axis=1, inplace=True)
Super_euro.drop('ISO3_CODE', axis=1, inplace=True)
Super_euro.drop('FID', axis=1, inplace=True)
gc.collect()

42

In [10]:
#pro_sel.sample(2)

# Data Visualization using Bokeh

In [11]:
import json
from bokeh.io import output_file, output_notebook
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,  
                          GroupFilter, CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure, save, show
from bokeh.models import Range1d
from bokeh.palettes import Viridis256 # Define color palettes
from bokeh.transform import transform
from bokeh.models import BasicTicker, PrintfTickFormatter


# First Visualization of an Empty map of Europe

In [12]:
# Name of the file.
output_file('Empty_Europe_2020.html', title='Empty_Europe_2020')
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = Super_euro.to_json())

p1 = figure(title = 'Europe 2020', 
           plot_height = 800 ,
           plot_width = 1300, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')

p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None

left, right, bottom, top = -13, 32, 30, 65
p1.x_range=Range1d(left, right)
p1.y_range=Range1d(bottom, top)

# Add patch renderer to figure.
states = p1.patches('xs','ys', source = geosource,
                   fill_color = None,
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1.5)

# Create hover tool
p1.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME_ENGL')]))
    

show(p1)

# Second Visualization of a coloured Europe Map

In [20]:
output_file('Custom_Coloured_Europe_2020.html', title='Custom_coloured_Europe_2020')

geosource = GeoJSONDataSource(geojson = Super_euro.to_json())

# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 50)
# Define custom tick labels for color bar.
tick_labels = {'0': '0', '3':'3',
 '5':'5', '10':'10','15':'15',
 '20':'20', '25':'25',
 '30':'30', '40':'40', '50':'50+'}
# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 10,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p2 = figure(title = 'Europe States - Selection Processes, 2019-2020', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None
#limit the map
left, right, bottom, top = -13, 45, 30, 72
p2.x_range=Range1d(left, right)
p2.y_range=Range1d(bottom, top)
# Add patch renderer to figure.
states = p2.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'Nu_Proc',
                                 'transform' : color_mapper},
                   line_color = 'gray',  
                   line_width = 0.25, 
                   fill_alpha = 1.5)

# Create hover tool
p2.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME_ENGL'),
                                 ('Number processes', '@Nu_Proc')]))
    
# Specify layout
p2.add_layout(color_bar, 'below')
show(p2)

# Third Visualization of Europe coloured Viridis

In [70]:
output_file('Viridis_col_Euro_2020.html', title='Viridis_col_Euro_2020')
geosource = GeoJSONDataSource(geojson = Super_euro.to_json())

# Define custom tick labels for color bar.
color_mapper = LinearColorMapper(palette = Viridis256, low = 0, high = 50)
#Define Color bar
color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

# Create figure object.
p2 = figure(title = 'Europe States - Selection Processes, 2019-2020', 
           plot_height = 800, plot_width = 1300, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None
#limit the map
left, right, bottom, top = -13, 32, 30, 65
p2.x_range=Range1d(left, right)
p2.y_range=Range1d(bottom, top)
# Add patch renderer to figure.
states = p2.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'Nu_Proc',
                                 'transform' : color_mapper},
                   line_color = 'gray',  
                   line_width = 0.25, 
                   fill_alpha = 1.5)

# Create hover tool
p2.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME_ENGL'),
                                 ('Number processes', '@Nu_Proc')]))
    
# Specify layout
p2.add_layout(color_bar, 'right')
show(p2)

In [59]:
######### Get x and y coordinates
###df4['geometry'] = geometry
#df4['x'] = [geometry.x for geometry in df4['geometry']]
#df4['y'] = [geometry.y for geometry in df4['geometry']]
#p_df = df4.drop('geometry', axis = 1).copy()


# GEOLOCALIZATION OF SELECTION PROCESSES IN Europe coloured Viridis

In [13]:
output_file('Vir_col_Geoprocess_2020.html', title='Vir_col_Geoprocess_2020')

geosource = GeoJSONDataSource(geojson=Super_euro.to_json())
source = ColumnDataSource(pro_sel)

#Italy_filters = [GroupFilter(column_name='State', group="Italy")]
#Italy_view = CDSView(source=source, filters=Italy_filters)
#Spain_filters = [GroupFilter(column_name='State', group="Spain")]
#Spain_view = CDSView(source=source, filters=Spain_filters)

#palette1 =['#000000', '#333333', '#666666', '#999999', '#cccccc', '#ffffff']
#color_mapper1 = LinearColorMapper(palette = palette1, low = 0, high = 15)

# Define custom tick labels for color bar.
color_mapper = LinearColorMapper(palette = Viridis256, low = 0, high = 50)
#Define Color bar
color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())
# Specify the selection tools to be made available
select_tools = ['pan, box_zoom, reset']
# Create figure object.
p = figure(title = 'Europe States - Selection Processes, 2019-2020', 
           plot_height = 800, plot_width = 1300, 
           toolbar_location = 'below',
           tools = select_tools)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#####limit the map
left, right, bottom, top = -13, 32, 30, 65
p.x_range=Range1d(left, right)
p.y_range=Range1d(bottom, top)
##### Ad#d patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'Nu_Proc',
                                 'transform' : color_mapper},
                   line_color = 'gray',  
                   line_width = 0.25, 
                   fill_alpha = 1.5)

circle = p.circle(x='Longitude', y='Latitude', 
                  source = source, 
                  #fill_color = {'field' :'Nu_Proc_Loc',
                                 #'transform' : color_mapper1},
                  #line_color = 'gray',
                  #line_width = 0.25) 
                  size= 'Size_Scale',
                  color='red')
         
         
#p.circle(x='Longitude' , y='Latitude', 
         #source = source , 
         #view = Italy_view,
         #legend_label='Italy',
         #selection_color='deepskyblue',
         #nonselection_color='lightgray',
         #color='red', 
         #size=7,
         #nonselection_alpha=0.5)

#p.square(x='Longitude' , y='Latitude', 
         #source = source , 
         #view = Spain_view,
         #legend_label='Spain',
         #selection_color='deepskyblue',
         #nonselection_color='lightgray',
         #color='red', 
         #size=7,
         #nonselection_alpha=0.5)

# Create hover tool
phover = HoverTool(renderers=[circle])
phover.tooltips=[('City', '@City'), ('Location Processes' , '@Nu_Proc_Loc')]
ghover = HoverTool(renderers=[states])
ghover.tooltips=[('State','@NAME_ENGL'), ('Country processes', '@Nu_Proc')]

p.add_tools(ghover)
p.add_tools(phover)
                 
#### Move the legend to the upper left corner
#p.legend.location = 'top_left'
#p.legend.click_policy="hide"

# Specify layout
p.add_layout(color_bar, 'right')
show(p)